# Importing Necessary Libraries

In [68]:
import pandas as pd
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np
import time

In [2]:
df = pd.read_csv('final_whiskey_data.csv')
df = df.drop('Unnamed: 0', axis = 1)
df.shape

(41018, 7)

In [3]:
pd.options.display.max_colwidth = 10000

In [81]:
df.head()


,rating,url,whiskey,User_ID,Whiskey_ID,price(euro),price(dollar)
0,88,https://www.whiskybase.com/whiskies/whisky/128130/1770-glasgow-single-malt,1770 Glasgow Single Malt,852,0,50.88,56.99
1,87,https://www.whiskybase.com/whiskies/whisky/128130/1770-glasgow-single-malt,1770 Glasgow Single Malt,459,0,50.88,56.99
2,86,https://www.whiskybase.com/whiskies/whisky/128130/1770-glasgow-single-malt,1770 Glasgow Single Malt,1205,0,50.88,56.99
3,85,https://www.whiskybase.com/whiskies/whisky/128130/1770-glasgow-single-malt,1770 Glasgow Single Malt,547,0,50.88,56.99
4,84,https://www.whiskybase.com/whiskies/whisky/128130/1770-glasgow-single-malt,1770 Glasgow Single Malt,562,0,50.88,56.99


In [5]:
df['whiskey'] = df['whiskey'].str.replace('-', ' ').str.title()

# Need to make a Dataframe with only User, Item, and Rating for Surprise

In [6]:
rec_df = df.drop(columns = ['url', 'price(euro)', 'price(dollar)'])
rec_df = rec_df[['User_ID', 'Whiskey_ID', 'rating']]

# DataFrame with unique WhiskeyID's to provide all the information from the recommender

In [7]:
rec_match = df[['url', 'whiskey', 'Whiskey_ID', 'price(dollar)', 'price(euro)']].drop_duplicates('Whiskey_ID')

In [8]:
rec_match.head()

,url,whiskey,Whiskey_ID,price(dollar),price(euro)
0,https://www.whiskybase.com/whiskies/whisky/128130/1770-glasgow-single-malt,1770 Glasgow Single Malt,0,56.99,50.88
6,https://www.whiskybase.com/whiskies/whisky/125010/a-dream-of-scotland-islay-cask-strength-bw,A Dream Of Scotland Islay Cask Strength Bw,1,89.59,79.99
53,https://www.whiskybase.com/whiskies/whisky/126951/a-dream-of-scotland-the-old-lady-of-islay-bw,A Dream Of Scotland The Old Lady Of Islay Bw,2,87.25,77.90
85,https://www.whiskybase.com/whiskies/whisky/129452/aberfeldy-1996-ca,Aberfeldy 1996 Ca,3,97.44,87.00
87,https://www.whiskybase.com/whiskies/whisky/131516/aberfeldy-1998,Aberfeldy 1998,4,319.20,285.00


In [9]:
rec_df.User_ID.max()

1222

# Fitting the model with our data

In [10]:
from surprise import Reader, Dataset
reader = Reader(rating_scale = (1,100))
data = Dataset.load_from_df(rec_df,reader)

In [11]:
from surprise.model_selection import train_test_split
from surprise import accuracy
# Splitting the data to see how accurate our model will be predicting a rating
trainset, testset = train_test_split(data, test_size=0.25, random_state = 100)

In [12]:
svd = SVD()
svd.fit(trainset)
preds = svd.test(testset)

In [13]:
# Rating the Model
accuracy.rmse(preds)

RMSE: 2.3577


2.3577444245702304

# Test function to see how well our model predicts

In [36]:
def recommend_whiskey(uid, iid, actual):
    pred = svd.predict(uid, iid, actual, verbose=True)
    match = rec_match.loc[rec_match['Whiskey_ID'] == iid]
    return match

In [37]:
recommend_whiskey(852,0, 88)

user: 852        item: 0          r_ui = 88.00   est = 89.20   {'was_impossible': False}


,url,whiskey,Whiskey_ID,price(dollar),price(euro)
0,https://www.whiskybase.com/whiskies/whisky/128130/1770-glasgow-single-malt,1770 Glasgow Single Malt,0,56.99,50.88


# Helper function to return recommendations to the user. Used in the whiskey_rater function below

In [50]:
## add the new ratings to the original ratings DataFrame
def get_recommendations(user_ratings, num_recs, priceUSD):
    new_ratings_df = rec_match.append(user_ratings,ignore_index=True)
    new_ratings_df = new_ratings_df[new_ratings_df['price(dollar)'] < priceUSD]
    new_ratings_df = new_ratings_df.drop(columns=['url', 'whiskey', 'price(dollar)', 'price(euro)'])

    #load in new df
    new_data = Dataset.load_from_df(new_ratings_df,reader)
    #create new svd object
    svd_new = SVD()
    #re fit the model
    svd_new.fit(new_data.build_full_trainset())

    # make predictions for the user
    list_of_whiskies = []
    for w_id in new_ratings_df['Whiskey_ID'].unique():
        list_of_whiskies.append((w_id, svd_new.predict(new_ratings_df['User_ID'].max(),w_id)[3]))

    # order the predictions from highest to lowest rated
    ranked_whiskies = sorted(list_of_whiskies, key=lambda x:x[1],reverse=True)

    rec_num = 1
    for i in ranked_whiskies[:num_recs]:
        recommended = rec_match[rec_match['Whiskey_ID'] == i[0]]
        print('Recommendation number:', rec_num)
        print('Whiskey: ' + recommended.values[0][1])
        print('Price: ' + str(recommended.values[0][-1]))
        print('URL: ' + recommended.values[0][0])
        print('\n')
        rec_num +=1



In [77]:
def whiskey_rater(df, num):
    userID = rec_df.User_ID.max()+1
    num_recs = input('How many recommendations would you like? Please enter a number from 1 to 10:\n')
    
    while int(num_recs) > 10:
        num_recs = input('You entered a number over 10. Please enter a number from 1 to 10 to continue. \n')
    priceUSD = input('Please enter your budget for a bottle of whiskey. $')
    rating_list = []
    
    while num > 0:
        whiskey = df[df['price(dollar)'] < int(priceUSD)].sample(1)
        print('\nPlease rate the following {} whiskies. \n'.format(num))
        print('Whiskey: ' + whiskey.values[0][2])
        print('Price: ' + str(whiskey.values[0][-1]))
        print('URL: ' + whiskey.values[0][1])
        rating = input('How do you rate this whiskey on a scale of 1-100, press n if you are not familiar with it. :\n')
            
        if rating == 'n':
            continue
        
        if int(rating) > 100:
            print('Rating must be below 100!')
            continue
        
        else:
            rating_one_whiskey = {'User_ID':userID,'Whiskey_ID': whiskey['Whiskey_ID'].values[0],'rating': rating}
            rating_list.append(rating_one_whiskey) 
            num -= 1
        time.sleep(.5)
    print('\n'+'-----Making Recommendations-----'+'\n')
    
    time.sleep(1.5)
    get_recommendations(rating_list, int(num_recs), int(priceUSD))
        

In [79]:
user_ratings= whiskey_rater(df, 5)

How many recommendations would you like? Please enter a number from 1 to 10:
5
Please enter your budget for a bottle of whiskey. $100

Please rate the following 5 whiskies. 

Whiskey: Deanston 2008 Ca
Price: 62.72
URL: https://www.whiskybase.com/whiskies/whisky/129558/deanston-2008-ca
How do you rate this whiskey on a scale of 1-100, press n if you are not familiar with it. :
60

Please rate the following 4 whiskies. 

Whiskey: Classic Of Islay Vintage 2019 Jw
Price: 56.0
URL: https://www.whiskybase.com/whiskies/whisky/127659/classic-of-islay-vintage-2019-jw
How do you rate this whiskey on a scale of 1-100, press n if you are not familiar with it. :
50

Please rate the following 3 whiskies. 

Whiskey: Glen Scotia 2006
Price: 95.2
URL: https://www.whiskybase.com/whiskies/whisky/128449/glen-scotia-2006
How do you rate this whiskey on a scale of 1-100, press n if you are not familiar with it. :
60

Please rate the following 2 whiskies. 

Whiskey: Kilkerran Heavily Peated
Price: 80.89
URL: